**This kernel uses Deepfm model from deepctr package**

**Deepfm :** [Deepfm using deepctr](https://deepctr-doc.readthedocs.io/en/v0.6.3/deepctr.models.deepfm.html)

Guo H, Tang R, Ye Y, et al. Deepfm: a factorization-machine based neural network for ctr prediction[J]. arXiv preprint arXiv:1703.04247, 2017.(https://arxiv.org/abs/1703.04247)

In [1]:
# !pip install --no-warn-conflicts -q deepctr
# !pip install --no-warn-conflicts -U -q scikit-learn==0.22.1

In [2]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import MissingIndicator, SimpleImputer, IterativeImputer, KNNImputer

In [3]:
from deepctr.inputs import  SparseFeat, DenseFeat, get_feature_names
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam,RMSprop
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import backend as K
from tensorflow.keras import callbacks
from tensorflow.keras import utils
from deepctr.models import DeepFM
import tensorflow.keras as keras
import tensorflow as tf
import pandas as pd
import numpy as np
import re
import warnings
warnings.simplefilter('ignore')

In [5]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [6]:
test["target"] = -1

In [7]:
data = pd.concat([train, test]).reset_index(drop=True)

In [8]:
data['null'] = data.isna().sum(axis=1)

In [9]:
sparse_features = [feat for feat in train.columns if feat not in ['id','target']]

In [10]:
for col in sparse_features:
    train_unique_values = set(train[col].dropna().unique())
    test_unique_values  = set(test[col].dropna().unique())

    symmetric_difference_values = train_unique_values.symmetric_difference(test_unique_values)
    if symmetric_difference_values:
        print(f'{len(symmetric_difference_values)} values in {col}, {symmetric_difference_values} Replaced with nan')
        data.loc[data[col].isin(symmetric_difference_values), col] = np.nan

1 values in nom_5, {'b3ad70fcb'} Replaced with nan
4 values in nom_6, {'ee6983c6d', 'f0732a795', '3a121fefb', 'a885aacec'} Replaced with nan
2 values in nom_9, {'3d19cd31d', '1065f10dd'} Replaced with nan


In [11]:
missing_indicators = MissingIndicator(sparse=False).fit_transform(data[sparse_features]).astype(np.int8)

In [12]:
missing_indicator_cols = [feat+'_ind' for feat in sparse_features]
for col in missing_indicator_cols:
    data[col] = 0
    data[col] = data[col].astype(np.uint8)
data[missing_indicator_cols] = MissingIndicator(sparse=False).fit_transform(data[sparse_features]).astype(np.int8)

In [13]:
for feat in sparse_features:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat].fillna('-1',).astype(str).values)

In [14]:
train = data[data.target != -1].reset_index(drop=True)
test  = data[data.target == -1].reset_index(drop=True)

In [15]:
fixlen_feature_columns = [SparseFeat(feat, data[feat].nunique()) for feat in (sparse_features+missing_indicator_cols)]

dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

In [16]:
def auc(y_true, y_pred):
    def fallback_auc(y_true, y_pred):
        try:
            return roc_auc_score(y_true, y_pred)
        except:
            return 0.5
    return tf.py_function(fallback_auc, (y_true, y_pred), tf.double)

In [17]:
class CyclicLR(keras.callbacks.Callback):

    def __init__(self, base_lr=0.001, max_lr=0.006, step_size=2000., mode='triangular',
                 gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1 / (2. ** (x - 1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma ** (x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        """Resets cycle iterations.
        Optional boundary/step size adjustment.
        """
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size
        self.clr_iterations = 0.

    def clr(self):
        cycle = np.floor(1 + self.clr_iterations / (2 * self.step_size))
        x = np.abs(self.clr_iterations / self.step_size - 2 * cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr - self.base_lr) * np.maximum(0, (1 - x)) * self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr - self.base_lr) * np.maximum(0, (1 - x)) * self.scale_fn(
                self.clr_iterations)

    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())

    def on_batch_end(self, epoch, logs=None):

        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1

        K.set_value(self.model.optimizer.lr, self.clr())


In [24]:
with tf.compat.v1.Session() as sess:
    devices = sess.list_devices()
devices

[_DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 268435456, 116913325137957555),
 _DeviceAttributes(/job:localhost/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 5788876685929312467),
 _DeviceAttributes(/job:localhost/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 9213960874773253742)]

In [20]:
target = ['target']
N_Splits = 50
Verbose = 2
Epochs = 9
SEED = 2020

In [21]:
oof_pred_deepfm = np.zeros((len(train), ))
y_pred_deepfm = np.zeros((len(test),))


skf = StratifiedKFold(n_splits=N_Splits, shuffle=True, random_state=SEED)
for fold, (tr_ind, val_ind) in enumerate(skf.split(train, train[target])):
    X_train, X_val = train[sparse_features+missing_indicator_cols].iloc[tr_ind], train[sparse_features+missing_indicator_cols].iloc[val_ind]
    y_train, y_val = train[target].iloc[tr_ind], train[target].iloc[val_ind]
    train_model_input = {name:X_train[name] for name in feature_names}
    val_model_input = {name:X_val[name] for name in feature_names}
    test_model_input = {name:test[name] for name in feature_names}
    model = DeepFM(linear_feature_columns, dnn_feature_columns, dnn_hidden_units=(256, 256), dnn_dropout=0.0, dnn_use_bn=False, task='binary')
    model.compile("adam", "binary_crossentropy", metrics=[auc], )
    es = callbacks.EarlyStopping(monitor='val_auc', min_delta=0.001, patience=4, verbose=Verbose, mode='max', baseline=None, restore_best_weights=True)
    sb = callbacks.ModelCheckpoint('./nn_model.w8', save_weights_only=True, save_best_only=True, verbose=Verbose)
    clr = CyclicLR(base_lr=0.00001 / 100, max_lr = 0.0001, 
                       step_size= int(1.0*(test.shape[0])/1024) , mode='exp_range',
                       gamma=1., scale_fn=None, scale_mode='cycle')
    history = model.fit(train_model_input, y_train,
                        validation_data=(val_model_input, y_val),
                        batch_size=256, epochs=Epochs, verbose=Verbose,
                        callbacks=[es, sb, clr],)
    model.load_weights('./nn_model.w8')
    val_pred = model.predict(val_model_input, batch_size=256)
    print(f"validation AUC fold {fold+1} : {round(roc_auc_score(y_val, val_pred), 5)}")
    oof_pred_deepfm[val_ind] = val_pred.ravel()
    y_pred_deepfm += model.predict(test_model_input, batch_size=256).ravel() / (N_Splits)
    K.clear_session()


Train on 588000 samples, validate on 12000 samples
Epoch 1/9

Epoch 00001: val_loss improved from inf to 0.40121, saving model to ./nn_model.w8
588000/588000 - 28s - loss: 0.4451 - auc: 0.7422 - val_loss: 0.4012 - val_auc: 0.7813
Epoch 2/9

Epoch 00002: val_loss improved from 0.40121 to 0.39942, saving model to ./nn_model.w8
588000/588000 - 23s - loss: 0.3939 - auc: 0.7920 - val_loss: 0.3994 - val_auc: 0.7844
Epoch 3/9

Epoch 00003: val_loss did not improve from 0.39942
588000/588000 - 23s - loss: 0.3912 - auc: 0.7958 - val_loss: 0.4001 - val_auc: 0.7833
Epoch 4/9

Epoch 00004: val_loss did not improve from 0.39942
588000/588000 - 23s - loss: 0.3904 - auc: 0.7968 - val_loss: 0.4009 - val_auc: 0.7820
Epoch 5/9

Epoch 00005: val_loss did not improve from 0.39942
588000/588000 - 23s - loss: 0.3901 - auc: 0.7971 - val_loss: 0.4005 - val_auc: 0.7824
Epoch 6/9
Restoring model weights from the end of the best epoch.

Epoch 00006: val_loss did not improve from 0.39942
588000/588000 - 23s - los

KeyboardInterrupt: 

In [ ]:
print(f"OOF AUC : {round(roc_auc_score(train.target.values, oof_pred_deepfm), 5)}")

In [ ]:
test_idx = test.id.values
submission = pd.DataFrame.from_dict({
    'id': test_idx,
    'target': y_pred_deepfm
})
submission.to_csv("submission.csv", index=False)
print("Submission file saved!")

In [ ]:
np.save('oof_pred_deepfm.npy',oof_pred_deepfm)
np.save('y_pred_deepfm.npy',    y_pred_deepfm)